
Customer Churn Prediction is the use of data and predictive analytics to foresee which customers are likely to leave, enabling businesses to take preventive actions and enhance customer retention.

# Notebook Content 

## AutoSearch Vs NN
- [1.0- Load Data and Imports](#1.0)
    - [1.1- Import require lib](#1.1)
- [2.0- EDA](#2.0)
    - [2.1- Illustrate the correlation or connection between different columns in the dataset.](#2.1)
- [3.0- Preprocessing Data and Load Model](#3.0)
    - [ 3.1- Preprocessing ](#3.1)
    - [ 3.2- AutoSearch model](#3.2)
    - [ 3.3- Make Prediction](#3.3)
- [4.0- NN Model](#4.0)
    - [ 4.1- Prepare the Data](#4.1)
    - [ 4.2- Create Neural Network Model](#4.2)
    - [ 4.3- Make Prediction](#4.3)


# [1.1- Import require lib](#1.1)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import IPython
import altair as alt 
import warnings
import torch 

from IPython.display import HTML , display , Image
from sklearn.preprocessing import OneHotEncoder , LabelEncoder , StandardScaler 
from sklearn.compose import ColumnTransformer  , make_column_transformer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import cross_val_score , cross_validate , train_test_split , RandomizedSearchCV , KFold
from sklearn.pipeline import make_pipeline ,Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , f1_score, make_scorer, recall_score 

import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import TensorDataset ,  DataLoader




In [ ]:
#Load dataset 

data_path = '/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv'

data      = pd.read_csv(data_path)
data.head()

In [ ]:
columns_list = data.columns.tolist()
columns_list

In [ ]:
data.isnull().sum()

# [2.0- EDA](#2.0)

In [ ]:
data.describe()

In [ ]:
data['Exited'].value_counts()

We have a class imbalance. Both classes seem importance here.
We will be going to use macro-average f1 score as our evaluation metric.

In [ ]:
metrics_score =  make_scorer(f1_score , average='macro')
score         = metrics_score

In [ ]:
data = data.drop(['RowNumber','CustomerId','Surname'] , axis=1)


As can be seen, column `Age` is the most relavent of colums. lets show the density of each type of `Gender` relavent with `Age` columns

# [2.1- Illustrate the correlation or connection between different columns in the dataset.](#2.1)

In [ ]:
Gender_Density = (alt.Chart(data[:5000])
              .transform_density(
                                  'Age' , 
                                   groupby = ['Exited' , 'Gender'] ,
                                   as_     = ['Age' , 'Density'] ,
                                   counts  = True,).mark_line().encode(
                                 x='Age',
                                 y = 'Density:Q',
                                 color= 'Exited:N',
                                 tooltip='Age').facet('Gender',
                                 title = 'Age Density'))
Gender_Density

# [3.0- Preprocessing Data and Load Model](#3.0)

In [ ]:
data.tail()

# [ 3.1- Preprocessing ](#3.1)


### spliting data into train and testsets 

In [ ]:
y_label = data['Exited']
data = data.drop(['Exited'] , axis=1)

X_train , X_test , y_train , y_test = train_test_split(data , y_label , test_size=0.2 , random_state=12)


In [ ]:
# categorical_columns = ['Geography' ,'Gender' ] 
# scaler_columns    = ['CreditScore' , 'Balance' , 'EstimatedSalary']

Encoder1 = LabelEncoder()
Encoder2 = LabelEncoder()
Scaler_model = StandardScaler()

X_train['Geography'] = Encoder1.fit_transform(X_train['Geography']).tolist()
X_train['Gender']    = Encoder2.fit_transform(X_train['Gender']).tolist()
#########################################
X_test['Geography'] = Encoder1.transform(X_test['Geography']).tolist()
X_test['Gender']    = Encoder2.transform(X_test['Gender']).tolist()

X_train = Scaler_model.fit_transform(X_train)
X_test  = Scaler_model.transform(X_test)
#########################################
X_train = pd.DataFrame(X_train , columns = columns_list[3:-1])
X_test = pd.DataFrame(X_test , columns = columns_list[3:-1])

In [ ]:
train_data =  X_train.copy()
train_data['Exited'] = y_train



fig , ax = plt.subplots(figsize=(12,12))
sns.heatmap(train_data.corr() , annot= True , ax=ax ,cmap= 'Greens' ).set(title='Feature Corelation' , xlabel='Columns' , ylabel='Columns')
plt.show()

In [ ]:
X_train.head()

In [ ]:
X_test.head()

# [ 3.2- AutoSearch model](#3.2)


## Load Model

in this section, I will use Auto Search technique to get the best model training on our data 
so let's dive in.

In [ ]:
!python3 -m pip install -q evalml==0.28.0  
####################################
!pip install fast_ml
from evalml.automl import AutoMLSearch

In [ ]:
# Use Algorithm 


EV_model = AutoMLSearch(X_train= X_train  , y_train =y_train , problem_type='binary' 
                        ,max_time = 70)
EV_model.search()

In [ ]:
EV_model.rankings

In [ ]:
EV_model.best_pipeline

# [ 3.3- Make Prediction](#3.3)

In [ ]:
test_prediction = EV_model.best_pipeline.predict(X_test)


submission = pd.DataFrame(test_prediction)

In [ ]:
submission.head()

In [ ]:
print(classification_report(y_test , test_prediction))

In [ ]:
print(confusion_matrix(y_test , test_prediction))

# [ NN Model ](#4.0)

# [ 4.1- Prepare the Data](#4.1)


In [ ]:
X_tr = torch.tensor(X_train.values , dtype = torch.float32) 
X_val= torch.tensor(X_test.values , dtype = torch.float32)
y_tr = torch.tensor(y_train.values , dtype = torch.float32)
y_val= torch.tensor(y_test.values , dtype = torch.float32)
##########
train = TensorDataset(X_tr,y_tr) 
test  = TensorDataset(X_val , y_val)
##########
train_loader = DataLoader(train , batch_size=60 , shuffle=True) 
test_loader  = DataLoader(test  , batch_size=60 , shuffle=True)

# [ 4.2- Create Neural Network Model](#4.2)
   

In [ ]:
# create Model

class NeuralNetwork(nn.Module):
    def __init__(self , len_f):
        super(NeuralNetwork , self).__init__()
        self.fc1= nn.Linear(len_f , 10)
        self.fc2= nn.Linear(10 ,128)
        self.fc3= nn.Linear(128 ,64)
        self.fc4= nn.Linear(64 ,2)
        self.norm1= nn.BatchNorm1d(10)
        self.norm2= nn.BatchNorm1d(128)
        self.norm3= nn.BatchNorm1d(64)
    def forward(self , X):
        out = F.relu(self.norm1(self.fc1(X)))
#         out = F.dropout(out)
        out = F.relu(self.norm2(self.fc2(out)))
#         out = F.dropout(out)
        out = F.relu(self.norm3(self.fc3(out)))
#         out = F.dropout(out)
        out = F.softmax(self.fc4(out))
        return out
        

In [ ]:
len_features = X_tr.shape[1]
NN_model = NeuralNetwork(len_features)
NN_model.train()

In [ ]:
# initialize features 

n_epochs  = 200 
lr        = 1e-5
optimizer = optim.Adam(params =  NN_model.parameters() , lr=lr)
ceriterion= nn.CrossEntropyLoss()
device    = 'cuda' if torch.cuda.is_available() else 'cpu'
device

### Training Model

In [ ]:
X_shape = X_tr.shape[1]
X_shape

In [ ]:
line_dash = '-'.join('' for _ in range(100))
for epoch in range(n_epochs):
    total_accuracy = 0 
    total_loss    = 0
    print(line_dash)
    print(f'epoch : {epoch+1}')
    for data , label in train_loader:
        data = data.view(-1,X_shape)
        label= label
        
        out  = NN_model(data)
        out_tensor = torch.tensor(out.argmax(dim=1) , dtype=torch.float32 ,requires_grad=True)
        loss = ceriterion(out_tensor ,label)
        
        # backward 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy = ((out_tensor ==label ).float().mean())
        
        total_accuracy += accuracy / len(train_loader) 
        total_loss += loss / len(train_loader)
    print(f'train accuracy :{total_accuracy}\t train loss :{total_loss}')
    test_accuracy = 0 
    test_loss     = 0
    
    with torch.no_grad():
        for data2 , label2 in test_loader:
            data2 = data2.view(-1,X_shape)
            label2= label2
            out2  = NN_model(data2)
            out2_tensor= torch.tensor(out2.argmax(dim=1) , dtype=torch.float32 , requires_grad=False)
            loss  = ceriterion(out2_tensor,label2)
            
            accuracy2 = ((out2_tensor == label2).float().mean())
            test_accuracy += accuracy2 / len(test_loader)
            test_loss += loss / len(test_loader)
        print(f'test accuracy :{test_accuracy}\t test loss :{test_loss}')
        print(line_dash)
        print('\n')
            
        

# [ 4.3- Make Prediction](#4.3)

In [ ]:
# make prediction 


NN_prediction = NN_model(X_val).argmax(dim=1)

val_prediction = torch.tensor(NN_prediction , dtype=torch.float32)
# NN_prediction[:50]

cal_accuracy = (val_prediction == y_val).float().sum()
accuracy     = cal_accuracy / len(X_val)
print(confusion_matrix(val_prediction , y_val))
print(f'Classification Report :\n{classification_report(val_prediction ,y_val)} ')

###  ML algorithm do best than Neural Network

If you examine the 'Classification Report' section (macro avg), you will notice that machine learning outperforms neural networks. This might be due to the imbalance in the data. Therefore, if you set achieving balance as your goal, you are likely to achieve higher accuracy than what you have observed.